In [25]:
import kfp
from kfp import dsl
import kfp.components as components

def download_dataset(): 
    from torchvision import datasets
    from torchvision import transforms
    from torch.utils.data import DataLoader
    import os
    download_root = '/MNIST_data/' 

    train_dataset = datasets.MNIST(root=download_root,
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True) 

    test_dataset = datasets.MNIST(root=download_root,
                            train=False,
                            transform=transforms.ToTensor(), 
                            download=True) 


    batch_size = 100 

In [26]:
def train():
  import torch.nn as nn
  import torch
  from torchvision import datasets
  from torchvision import transforms
  from torch.utils.data import DataLoader
  import numpy as np
  import json
  import os

  class Net(nn.Module): 
      
      def __init__(self):
          super(Net, self).__init__()
          self.fc1 = nn.Linear(784,100) 
          self.relu = nn.ReLU()
          self.fc2 = nn.Linear(100,100) 
          self.fc3 = nn.Linear(100,10) 

      def forward(self, x): 
          x1 = self.fc1(x)
          x2 = self.relu(x1)
          x3 = self.fc2(x2)
          x4 = self.relu(x3)
          x5 = self.fc3(x4)

          return x5
  
  download_root = './MNIST_data'


  train_dataset = datasets.MNIST(root=download_root,
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True) 
                          
  test_dataset = datasets.MNIST(root=download_root,
                          train=False,
                          transform=transforms.ToTensor(), 
                          download=True) 


  batch_size = 100
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True) 

  model = Net() 
  loss_function = nn.CrossEntropyLoss() 

  optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.9)
  epochs = 1

  best_accuracy = 0
  model.zero_grad() 
  
  for epoch in range(epochs):
    
    model.train() 
    train_accuracy = 0 
    train_loss = 0 

    for images, labels in train_loader:
      images = images.reshape(batch_size,784)
      image = model(images)
      loss = loss_function(image,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      prediction = torch.argmax(image,1)
      correct = (prediction == labels)
      train_accuracy+= correct.sum().item() / len(train_dataset)
      train_loss += loss.item() / len(train_loader)

    model.eval() 
    val_accuracy = 0 
    val_loss = 0 

    for images,labels in test_loader:
      images = images.reshape(batch_size,784)
      image = model(images)
      loss = loss_function(image,labels)
      
      correct = (torch.argmax(image,1) == labels)
      val_accuracy += correct.sum().item() / len(test_dataset)
      val_loss += loss.item() / len(test_loader)
    
    print(f'epoch: {epoch}/{epochs} train_loss: {train_loss:.5} train_accuracy: {train_accuracy:.5} val_loss: {val_loss:.5} val_accuracy: {val_accuracy:.5}')

    if best_accuracy < val_accuracy: 
      best_accuracy = val_accuracy
      best_val_loss = val_loss
      torch.save(model.state_dict(),'./best_model.pt')
      print(f"===========> Save Model(Epoch: {epoch}, Accuracy: {best_accuracy:.5})")
    

    print("--------------------------------------------------------------------------------------------")
  
  metrics = {
        'metrics': [{
            'name': 'accuracy-score',
            'numberValue':  best_accuracy,
            'format': "PERCENTAGE",
        }]
    }
  
  with open('./mlpipeline-metrics.json','w') as f:
    json.dump(metrics,f)

  print("best_model uploaded to pvc!")

In [27]:
data_comp = components.create_component_from_func(download_dataset,base_image='public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-pytorch:v1.5.0')
train_comp = components.create_component_from_func(train,base_image='public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-pytorch:v1.5.0')

In [28]:
@dsl.pipeline(
    name='practice-pipeline'
)
def test():
    step1 = data_comp()
    step2 = train_comp()

    step2.after(step1)

if __name__ == '__main__':
    import kfp.compiler as compiler
    compiler.Compiler().compile(test,"test.yaml")